## 1: nonlocal variables

`python3`中加入了`nonlocal`这个关键词，`nonlocal`将范围覆盖添加到内部范围。

In [5]:
def counter():
    num = 0
    def increment():
        num += 1
        return num
    return increment
c = counter()
# c()
## 运行c()会报错，局部变量没有分配

def counter_right():
    num = 0
    def incrementer():
        nonlocal num
        num += 1
        return num
    return incrementer

c = counter_right()
print(c())
print(c())
print(c())


1
2
3


`nonlocal`允许你在外部范围分配变量，而不是在全局范围分配变量。因此不能在`counter`函数中使用`nonlocal`，只能在嵌套的函数中使用`nonlocal`

## 2: 全局变量

在`python`中，当变量出现在赋值语句的左边或其他绑定发生时，函数中的这个变量才被认为时局部变量，否则将在封闭的环境中寻找直到全局变量。

In [10]:
x = 'hi'

# def read_x():
#     print(x)
    
# read_x()

# def read_y():
#     print(y)
    
# read_y()

def read_x_local_fail():
    if False:
        x = 'Hey'
    print(x)
read_x_local_fail()  # 报错

UnboundLocalError: local variable 'x' referenced before assignment

内部的赋值会覆盖到外部的同名变量

In [12]:
x = 'hi'
def change_local_x():
    x = 'bye'
    print(x)
change_local_x()
print(x)

bye
hi


#### 总结

- 1.如果发现`global x`, 那么`x`时全局变量
- 2.如果发现`nonlocal x`,那么`x`时属于封闭环境，既不是局部的也不是全局的
- 3.如果发现$x = 5$或者`for x in range(2)`或其他的绑定，`x`是局部变量
- 4.否者`x`属于封闭环境（函数范围，全局范围，或局部部件）

## 3. 局部变量

如果名称绑定在函数内部，那么默认只能在函数中使用

In [13]:
def foo():
    a = 5
    print(a)
    
print(a)

NameError: name 'a' is not defined

控制流结构对范围没有影响（`except`除外），但是访问没有分配到变量是错误的。

In [14]:
def foo():
    if True:
        x = 5
    print(x)
    
b = 3
def bar():
    if False:
        b = 5
    print(b)
    
bar()

UnboundLocalError: local variable 'b' referenced before assignment

## 4：删除命令


In [1]:
x = 5
print(x)
del x
print(x)

5


NameError: name 'x' is not defined

类的定义方式可能使命令无效

`del v.name`会调用`v.__delattr__(name)`

其目的是使属性名不可用

`del v[item]`会调用`v.__delitem__(item)`

In [2]:
class A:
    pass
a = A()
a.x = 7
print(a.x)     # 输出7
del a.x
print(a.x)    # Error
del v[item]

7


AttributeError: 'A' object has no attribute 'x'

In [4]:
x = {'a':1, 'b':2}
print(x)
del x['a']
print(x)

## 会调用   v.__delitem__(item)

{'a': 1, 'b': 2}
{'b': 2}


In [5]:
x = [0,1,2,3,4]
print(x)
del x[1:2]
print(x)
## 会调用   v.__delslice__(a, b)

[0, 1, 2, 3, 4]
[0, 2, 3, 4]


## 5: 函数在查找名称时跳过类作用域

在定义的时候类有一个局部范围，但是在类中的函数在查找名称的时候不使用这个范围。因为`lambdas`是一个函数，并且理解是使用函数作用域实现的，这可能会导致一些令人惊讶的行为。

In [12]:
a = 'global'
# b = (a for i in range(10))         # 是一个迭代器
# print(b)
# print(next(b))
class Fred:
    a = 'class'
    b = (a for i in range(10))  # function scope函数作用域
    c = [a for i in range(10)]   # function scope函数作用域
    d = a
    e = lambda: a   # function scope
    f = lambda a = a: a   # default argument uses class scope
    
    @staticmethod
    def g():        # function scope
        return a
    
print(Fred.a)
print(next(Fred.b))
print(Fred.c[0])
print(Fred.d)
print(Fred.e())
print(Fred.f())
print(Fred.g())

class
global
global
class
global
class
global


In [14]:
class A:
    a = 42
    b = list(a + i for i in range(10))

## 6: 局部和全局

In [17]:
foo = 2  # global
def func():
    bar = 1  # local
    print(bar)
    print(foo)
    
func()

1
2


In [19]:
foo = 1
def func():
    bar = 2
    print(globals().keys())   # 输出所有的全局变量
    print(locals().keys())    # 输出所有的局部变量
    
func()

dict_keys(['__name__', '__doc__', '__package__', '__loader__', '__spec__', '__builtin__', '__builtins__', '_ih', '_oh', '_dh', 'In', 'Out', 'get_ipython', 'exit', 'quit', '_', '__', '___', '_i', '_ii', '_iii', '_i1', '_i2', 'A', 'a', '_i3', '_i4', 'x', '_i5', '_i6', '_i7', 'Fred', '_i8', '_i9', 'b', '_i10', '_i11', '_i12', '_i13', '_i14', '_i15', 'foo', 'func', '_i16', '_i17', '_i18', '_i19'])
dict_keys(['bar'])


- 当名字发生冲突的时候

In [22]:
foo = 1
def func():
    foo = 2
    print(foo)
    print(globals()['foo'])
    print(locals()['foo'])
    
func()

2
1
2


- 用`local`将局部变量更改为全局变量

In [24]:
foo = 1
def func():
    global foo
    foo = 2
    
func()
print(foo)          # 将全局变量更改了

2


- 作用域是对整个函数定义的

当函数中定义了局部变量是，将不会找到同名的全局变量

In [26]:
foo = 1

def func1():
    print(foo)
    
def func2():
    print(foo)       # 在后面foo又被定义为了局部变量
    
    foo = 7
    print(foo)
func1()
func2()

1


UnboundLocalError: local variable 'foo' referenced before assignment

In [29]:
foo = 1

def func():
    # 开始foo是一个全局变量
    global foo
    foo = 7  # 全局变量被修改
    
    print(foo)
    
    print(globals()['foo'])
    
#     global foo       # 可以在函数中的任意地方        ### 但是不能在认为地方啊
    print(foo)
func()

7
7
7


- 函数中的函数

在函数中的嵌套函数有很多等级，但是在任何函数中只有对函数的局部变量和全局变量，没有中间变量


In [39]:
foo = 1
def f1():
    bar = 1  # 仅仅是f1中的local
    
    def f2():
        baz = 2
        # foo 是全局变量，baz 是局部变量
        # bar 不在这两个范围内
        print(locals().keys())
        print('bar' in locals())
        print('bar' in globals())
    f2()
    
    def f3():
        baz = 3
        print(bar)
        print(locals().keys())
        print('bar' in locals())
        print('bar' in globals())
    f3()
    
    def f4():
        bar = 4
        baz = 4
        print(bar)
        print(locals().keys())
        print('bar' in locals())
        print('bar' in globals())
    f4()    
f1()



dict_keys(['baz'])
False
False
1
dict_keys(['baz', 'bar'])
True
False
4
dict_keys(['bar', 'baz'])
True
False


- `global` vs `nonlocal`(python 3 only)

In [40]:
foo = 0  # global

def f1():
    foo = 1   # local in f1
    
    def f2():
        foo = 2  # local in f2
        
        def f3():
            foo = 3  # local in f3
            print(foo)
            foo = 30   # 修改在f3中的foo
        f3()
            
        def f4():
            global foo
            print(foo)
            foo = 100
            
        f4()
    f2()
print(foo)    
f1()
print(foo)


0
3
0
100


In [41]:
def f1():
    def f2():
        foo = 2   # local in f2
        
        def f3():
            nonlocal foo  # foo from  f2, nearest enclosing scope
            print(foo)
            foo = 20  # modifies foo from f2
        f3()
        
    f2()
    
f1()

2


In [42]:
x = 5
x += 7
for x in iterable: pass

NameError: name 'iterable' is not defined

In [43]:
n = 5
'Greater than 2' if n > 2 else 'Smaller than or equal to 2'

'Greater than 2'

In [5]:
if not 0.0:
    print('yes')
    
if not {}:
    print('yes')

yes
yes


- None
- False
- 0; 0.0; 0.00
- 空序列 '';""; (); []
- 空映射: {}

都被认为是否

In [6]:
a = 1
def foo():
    a += 1
    print(a)
    
if foo():
    print('wrong')

UnboundLocalError: local variable 'a' referenced before assignment

In [15]:
1 and 2
1 and 0
1 and 'hello world'
'' and 'Pancakes'
'' and []


''

如果所有的都是`ture`，那`and`操作返回最后一个值,否则返回第一个`False`的那个值。

如果所有的都是`False`，那么`or`操作返回最后一个值，否则返回第一个`true`的值

In [13]:
1 or 2
None or 1
0 or []

[]

## lazy evaluation

当你使用这种方法的时候，记住这种评价是懒惰的。函数不会被执行，并且被认为是`false`

In [18]:
def print_me():
    print('i am here')
    
print_me() and 0
0 and print_me()

if not print_me():
    print('yes')

i am here
i am here
yes


## testing for multiple conditons

使用多重比较的时候常犯的错误

In [22]:
a = 1
b = 6
if a and b > 2:
    print('yes')
else:
    print('no')
    
if a > 2 and b > 2:
    print('yes')
else:
    print('no')
    
if a == 3 or a == 4 or a == 6:
    print('yes')
else:
    print('no')
    
## 规范的写法
if a in (3, 4, 5):
    print('yes')
else:
    print('no')

yes
no
no
no


## 使用`cmp`函数得到两个对象的比较结果

在python3中`cmp`函数被移除了

## 如果对象是`none`， 就给他赋值

In [29]:
import datetime
# aDate = None
# if aDate is None:            ## 使用is none 比使用 == none 更加的python    
#     aDate = datetime.date.today()
    

if not aDate:
    aDate = datetime.date.today()
    
aDate = aDate or datetime.date.today()